In [213]:
import pandas as pd
import re
import string
import numpy as np
import spacy

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

import nltk
import gensim
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV

In [120]:
train = pd.read_csv("E:\\backup\\OneDrive\\Desktop\\Data Science\\Project\\Kaggle\\train.csv")

In [121]:
test = pd.read_csv("E:\\backup\\OneDrive\\Desktop\\Data Science\\Project\\Kaggle\\test.csv")

In [122]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [123]:
test = test.drop(['location','id'], axis=1)

In [124]:
test

,keyword,text
0,NaN,Just happened a terrible car crash
1,NaN,"Heard about #earthquake is different cities, s..."
2,NaN,"there is a forest fire at spot pond, geese are..."
3,NaN,Apocalypse lighting. #Spokane #wildfires
4,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...
3258,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,NaN,Storm in RI worse than last hurricane. My city...
3260,NaN,Green Line derailment in Chicago http://t.co/U...
3261,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [125]:
y=train['target']

In [127]:
train = train.drop(['location','id','target'], axis=1)

In [128]:
train

,keyword,text
0,NaN,Our Deeds are the Reason of this #earthquake M...
1,NaN,Forest fire near La Ronge Sask. Canada
2,NaN,All residents asked to 'shelter in place' are ...
3,NaN,"13,000 people receive #wildfires evacuation or..."
4,NaN,Just got sent this photo from Ruby #Alaska as ...
...,...,...
7608,NaN,Two giant cranes holding a bridge collapse int...
7609,NaN,@aria_ahrary @TheTawniest The out of control w...
7610,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,NaN,Police investigating after an e-bike collided ...


In [129]:
test.isnull().sum()

keyword    26
text        0
dtype: int64

In [130]:
train.isnull().sum()

keyword    61
text        0
dtype: int64

In [131]:
train['keyword'].fillna(train['keyword'].mode()[0],inplace=True)

In [132]:
test['keyword'].fillna(test['keyword'].mode()[0],inplace=True)

In [133]:
train.head()

,keyword,text
0,fatalities,Our Deeds are the Reason of this #earthquake M...
1,fatalities,Forest fire near La Ronge Sask. Canada
2,fatalities,All residents asked to 'shelter in place' are ...
3,fatalities,"13,000 people receive #wildfires evacuation or..."
4,fatalities,Just got sent this photo from Ruby #Alaska as ...


In [134]:
train['text']=train.text.str.replace('[^a-zA-Z0-9\s#]', '',regex=True)
train['keyword']=train.keyword.str.replace('[^a-zA-Z0-9\s]', '',regex=True)

In [135]:
test['text']=test.text.str.replace('[^a-zA-Z0-9\s#]', '',regex=True)
test['keyword']=test.keyword.str.replace('[^a-zA-Z0-9\s]', '',regex=True)

In [136]:
train['text'] = train['text'].str.replace('[^\w\s]','')

C:\Users\HP\AppData\Local\Temp/ipykernel_6232/3276939684.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['text'] = train['text'].str.replace('[^\w\s]','')


In [137]:
test['text'] = test['text'].str.replace('[^\w\s]','')

C:\Users\HP\AppData\Local\Temp/ipykernel_6232/1211124082.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test['text'] = test['text'].str.replace('[^\w\s]','')


In [138]:
train.head()

,keyword,text
0,fatalities,Our Deeds are the Reason of this earthquake Ma...
1,fatalities,Forest fire near La Ronge Sask Canada
2,fatalities,All residents asked to shelter in place are be...
3,fatalities,13000 people receive wildfires evacuation orde...
4,fatalities,Just got sent this photo from Ruby Alaska as s...


In [139]:
train.iloc[1000:1050,:]

,keyword,text
1000,body20bagging,OVOFest Drake straight body bagging Meek on th...
1001,body20bagging,Im not a Drake fan but I enjoy seeing him body...
1002,body20bagging,amaramin3 Meek is definitely capable of body b...
1003,body20bagging,Gates not body bagging nobody niggas in br rea...
1004,body20bagging,WWE 2k15 MyCareer EP18 Tyrone body bagging dud...
1005,body20bagging,I did another one I did another one You still ...
1006,body20bagging,Drake Body Bagging Meek He must of hit a sensi...
1007,body20bagging,Good diss bad beat and flow Mark my words Meek...
1008,body20bagging,Drake is body bagging meek meanwhile hes on to...
1009,body20bagging,MzGracieBaby for the record im jumpin out the ...


In [140]:
train['keyword']=train['keyword'].str.replace('[/%20/]', " ")

C:\Users\HP\AppData\Local\Temp/ipykernel_6232/1043633802.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['keyword']=train['keyword'].str.replace('[/%20/]', " ")


In [141]:
test['keyword']=test['keyword'].str.replace('[/%20/]', " ")

C:\Users\HP\AppData\Local\Temp/ipykernel_6232/3048059646.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test['keyword']=test['keyword'].str.replace('[/%20/]', " ")


In [142]:
test

,keyword,text
0,deluged,Just happened a terrible car crash
1,deluged,Heard about earthquake is different cities sta...
2,deluged,there is a forest fire at spot pond geese are ...
3,deluged,Apocalypse lighting Spokane wildfires
4,deluged,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...
3258,deluged,EARTHQUAKE SAFETY LOS ANGELES SAFETY FASTENER...
3259,deluged,Storm in RI worse than last hurricane My citya...
3260,deluged,Green Line derailment in Chicago httptcoUtbXLc...
3261,deluged,MEG issues Hazardous Weather Outlook HWO httpt...


In [143]:
train.text[100]

'NorwayMFA Bahrain police had previously died in a road accident they were not killed by explosion httpstcogFJfgTodad'

In [144]:
train['text'] = train['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [145]:
test['text'] = test['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [146]:
train.text[150]

'mickinyman TheAtlantic That or they might be killed in an airplane accident in the night a car wreck Politics at its best'

In [147]:
tfidf = TfidfVectorizer(tokenizer=word_tokenize, token_pattern=None)

train_texts=tfidf.fit_transform(train['text'])
train_texts=pd.DataFrame(train_texts.toarray(),columns=tfidf.get_feature_names_out())

test_texts=tfidf.fit_transform(test['text'])
test_texts=pd.DataFrame(test_texts.toarray(),columns=tfidf.get_feature_names_out())

train_texts,test_texts=train_texts.align(test_texts,join='left',axis=1)

In [148]:
test_texts.fillna(0,inplace=True)
test_texts.isnull().sum().sum()

0

In [149]:
ohe_train_keyword=pd.get_dummies(train['keyword'])
ohe_test_keyword=pd.get_dummies(test['keyword'])
train_keyword,test_keyword=ohe_train_keyword.align(ohe_test_keyword, join='left', axis=1)

In [150]:
test_keyword.fillna(0,inplace=True)
test_keyword.isnull().sum().sum()

0

In [151]:
final_train_df=pd.concat([train_texts,train_keyword],axis=1)
final_test_df=pd.concat([test_texts,test_keyword],axis=1)

In [152]:
final_train_df

,0,0011,001116,0025,005225,010156,010217,0104,010401,0106,...,weapons,whirlwind,wild fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.449048,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
X=final_train_df

In [169]:
X

,0,0011,001116,0025,005225,010156,010217,0104,010401,0106,...,weapons,whirlwind,wild fires,wildfire,windstorm,wounded,wounds,wreck,wreckage,wrecked
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.449048,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 0)

In [159]:
lr = LogisticRegression()
model = lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

confussion matrix
[[1146  192]
 [ 275  671]]


Accuracy of Logistic Regression: 79.55341506129598 

              precision    recall  f1-score   support

           0       0.81      0.86      0.83      1338
           1       0.78      0.71      0.74       946

    accuracy                           0.80      2284
   macro avg       0.79      0.78      0.79      2284
weighted avg       0.79      0.80      0.79      2284



In [160]:
nb = GaussianNB()
nb.fit(X_train,y_train)
nbpred = nb.predict(X_test)
nb_conf_matrix = confusion_matrix(y_test, nbpred)
nb_acc_score = accuracy_score(y_test, nbpred)
print("confussion matrix")
print(nb_conf_matrix)
print("\n")
print("Accuracy of Naive Bayes model:",nb_acc_score*100,'\n')
print(classification_report(y_test,nbpred))

confussion matrix
[[687 651]
 [213 733]]


Accuracy of Naive Bayes model: 62.17162872154116 

              precision    recall  f1-score   support

           0       0.76      0.51      0.61      1338
           1       0.53      0.77      0.63       946

    accuracy                           0.62      2284
   macro avg       0.65      0.64      0.62      2284
weighted avg       0.67      0.62      0.62      2284



In [161]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
rf_predicted = rf.predict(X_test)
rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
rf_acc_score = accuracy_score(y_test, rf_predicted)
print("confussion matrix")
print(rf_conf_matrix)
print("\n")
print("Accuracy of Random Forest:",rf_acc_score*100,'\n')
print(classification_report(y_test,rf_predicted))

confussion matrix
[[1248   90]
 [ 366  580]]


Accuracy of Random Forest: 80.03502626970229 

              precision    recall  f1-score   support

           0       0.77      0.93      0.85      1338
           1       0.87      0.61      0.72       946

    accuracy                           0.80      2284
   macro avg       0.82      0.77      0.78      2284
weighted avg       0.81      0.80      0.79      2284



In [163]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predicted = knn.predict(X_test)
knn_conf_matrix = confusion_matrix(y_test, knn_predicted)
knn_acc_score = accuracy_score(y_test, knn_predicted)
print("confussion matrix")
print(knn_conf_matrix)
print("\n")
print("Accuracy of K-NeighborsClassifier:",knn_acc_score*100,'\n')
print(classification_report(y_test,knn_predicted))

confussion matrix
[[1077  261]
 [ 291  655]]


Accuracy of K-NeighborsClassifier: 75.83187390542906 

              precision    recall  f1-score   support

           0       0.79      0.80      0.80      1338
           1       0.72      0.69      0.70       946

    accuracy                           0.76      2284
   macro avg       0.75      0.75      0.75      2284
weighted avg       0.76      0.76      0.76      2284



In [164]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.get_n_leaves()
dt_predicted = dt.predict(X_test)
dt_conf_matrix = confusion_matrix(y_test, dt_predicted)
dt_acc_score = accuracy_score(y_test, dt_predicted)
print("confussion matrix")
print(dt_conf_matrix)
print("\n")
print("Accuracy of DecisionTreeClassifier:",dt_acc_score*100,'\n')
print(classification_report(y_test,dt_predicted))

confussion matrix
[[1058  280]
 [ 342  604]]


Accuracy of DecisionTreeClassifier: 72.76707530647985 

              precision    recall  f1-score   support

           0       0.76      0.79      0.77      1338
           1       0.68      0.64      0.66       946

    accuracy                           0.73      2284
   macro avg       0.72      0.71      0.72      2284
weighted avg       0.73      0.73      0.73      2284



In [172]:
svc =  SVC()
svc.fit(X_train, y_train)
svc_predicted = svc.predict(X_test)
svc_conf_matrix = confusion_matrix(y_test, svc_predicted)
svc_acc_score = accuracy_score(y_test, svc_predicted)
print("confussion matrix")
print(svc_conf_matrix)
print("\n")
print("Accuracy of Support Vector Classifier:",svc_acc_score*100,'\n')
print(classification_report(y_test,svc_predicted))

confussion matrix
[[1138  200]
 [ 299  647]]


Accuracy of Support Vector Classifier: 78.1523642732049 

              precision    recall  f1-score   support

           0       0.79      0.85      0.82      1338
           1       0.76      0.68      0.72       946

    accuracy                           0.78      2284
   macro avg       0.78      0.77      0.77      2284
weighted avg       0.78      0.78      0.78      2284



In [173]:
model_accuracy = pd.DataFrame({'Model': ['Logistic Regression','Naive Bayes','Random Forest',
                    'K-Nearest Neighbour','Decision Tree','Support Vector Machine'], 'Accuracy': [lr_acc_score*100,
                    nb_acc_score*100,rf_acc_score*100,knn_acc_score*100,dt_acc_score*100,svc_acc_score*100]})
model_accuracy

,Model,Accuracy
0,Logistic Regression,79.553415
1,Naive Bayes,62.171629
2,Random Forest,80.035026
3,K-Nearest Neighbour,75.831874
4,Decision Tree,72.767075
5,Support Vector Machine,78.152364


In [174]:
scores = cross_val_score(rf, final_train_df, y, cv=5)
scores.mean()

0.6699123298214069

In [175]:
train_results=rf.predict(final_train_df)

In [176]:
f1_train=f1_score(y,train_results)
print(f'F1 score on train set {f1_train}')

F1 score on train set 0.9182692307692307


In [177]:
test_results=rf.predict(final_test_df)

In [180]:
sample_sub=pd.read_csv('C:\\Users\\HP\\Downloads\\sample_submission.csv')

In [181]:
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':test_results})
sub.to_csv('submission1.csv',index=False)